Some $\LaTeX$ abbreviations:
$$
\newcommand{\pd}[2]{\frac{\partial #1}{\partial #2}}
$$

In [1]:
%pylab inline
%config InlineBackend.figure_format = 'retina'

Populating the interactive namespace from numpy and matplotlib


In [3]:
import astropy.cosmology as cosmo
from astropy.cosmology import Planck15
import astropy.units as u
import h5py

Let's try to estimate the precision of our measurement of $w$ by "collapsing" our 1000s of measurements of mass and distances into a single effective measurement at a "pivot" redshift of $z \sim 0.5$.  We have 
$$
d_L\left( z \mid H_0, \Omega_M, w \right) = \left( 1 + z \right) \int_0^z \mathrm{d} z' \, \frac{c}{H\left( z' \mid H_0, \Omega_M, w \right)},
$$
where 
$$
H\left( z' \mid H_0, \Omega_M, w \right) = H_0 \sqrt{\Omega_M \left( 1+z\right)^3 + \left(1 - \Omega_M \right) \left( 1 + z \right)^{3(1+w)}} \equiv H_0 E\left( z \mid \Omega_M, w \right).
$$

We collapse the population down to a single measurement: $d_L = \tilde{d}_L \pm \sigma_{d_L}$, $z = \tilde{z} \pm \sigma_z$ at a pivot redshift $\tilde{z} \simeq 0.5$ (so $\tilde{d}_L = d_L\left( \tilde{z} \mid \tilde{H}_0, \tilde{\Omega}_M, \tilde{w} \right)$ with $\tilde{H}_0 \simeq 67.74$, etc).  Here $\sigma_{d_L}$ and $\sigma_z \simeq \sigma_{\ln m_1}$ come from the measurement accuracy of the observations, either by combining the uncertainties from all the measurements via 
$$
\sigma_x^2 = \left[ \sum_{i} \left(\sigma_x^{(i)}\right)^{-2} \right]^{-1}
$$
or by taking a typical measurement uncertainty and scaling by $1/\sqrt{N}$, and $\sigma_{\ln H_0}$ and $\sigma_{\Omega_M}$ are the uncertainties in the prior measurements of these quantities ($\sim 1\%$ in both cases).  Then, assuming Gaussian priors on $H_0$ and $\Omega_M$, the posterior looks like 
$$
\log \pi \sim -\frac{1}{2} \left[ \left(\frac{\tilde{d}_L - d_L\left( z \mid H_0, \Omega_M, w \right)}{\sigma_{d_L}} \right)^2 + \left( \frac{\tilde{z} - z}{\sigma_z} \right)^2 + \left(\frac{\tilde{H}_0 - H_0}{\sigma_{H_0}}\right)^2 + \left( \frac{\tilde{\Omega}_M - \Omega_M}{\sigma_{\Omega_M}} \right)^2 \right].
$$
The gradient of the posterior vanishes when all parameters take on their tilde-d values; the Hessian (aka Fisher matrix) at this point is given by 
$$
\mathbf{F}_{ij} = - \mathbf{\nabla}_{i} \mathbf{\nabla}_{j} \log \pi,
$$
where $i = \left\{ H_0, \Omega_M, w, z \right\}$ ranges over the parameters in the system.  The covarance matrix of a Gaussian approximation to the posterior is given by $\mathbf{C} = \mathbf{F}^{-1}$; the diagonal entries in the covariance matrix give the variance in the corresponding parameter after marginalizing out the others.  

Because the derivatives in this expression are evaluated at the tilde-d value of the parameters, we can actually write the Fisher matrix as an outer product of gradients of $d_L\left( z \mid H_0, \Omega_M, w \right)$, $z$, $H_0$, and $\Omega_M$ (these latter three gradients are trivial identities).  The relevant derivatives of $d_L$ and their values at the fiducial parameters are 
$$
\pd{\ln d_L}{z} = \frac{1}{1+z} + \frac{\left(1+z\right) d_H}{d_L} \simeq 2.9,
$$
$$
\pd{\ln d_L}{\ln H_0} = -1,
$$
$$
\pd{\ln d_L}{\Omega_M} = \frac{\int_0^z \mathrm{d}z' \, \frac{\left( 1 + z' \right)^3 - \left(1+z'\right)^{3(1+w)}}{2 E^{3}\left(z'\right)}}{\int_0^z \mathrm{d}z' \, \frac{1}{E\left(z'\right)}} \simeq 0.34,
$$
and
$$
\pd{\ln d_L}{w} = \frac{\int_0^z \mathrm{d} z' \, \frac{3 \left( 1 - \Omega_M \right) \ln\left( 1 + z'\right) \left( 1 + z' \right)^{3(1+w)}}{2 E^3\left( z' \right)}}{\int_0^z \mathrm{d} z' \frac{1}{E\left( z' \right)}} \simeq 0.16
$$

In [17]:
z_pivot = 0.5
dlndLdz = 1/(1+z_pivot) + (1+z_pivot)*Planck15.hubble_distance/Planck15.luminosity_distance(z_pivot)
print(dlndLdz)

2.9414005273015023


In [18]:
zs = expm1(linspace(log(1), log(1+z_pivot), 100))
Ezs = Planck15.efunc(zs)
numer = trapz(((1+zs)**3-1)/(2*Ezs**3), zs)
denom = trapz(1/Ezs, zs)
dlndLdOm = numer/denom
print(dlndLdOm)

0.34337926212429837


In [19]:
numer = trapz(3*(1-0.3)*log1p(zs)/(2*Ezs**3), zs)
dlndLdw=numer/denom
print(dlndLdw)

0.15621504604988443


Now we just need to compute the $\sigma$s.  The only tricky bit is that only those masses that are sufficiently close to the upper boundary contribute.  For the moment let's leave that out, and just estimate.

In [16]:
with h5py.File('observations.h5', 'r') as f:
    sigma_lndl = 1.0/sqrt(sum(1/var(log(f['posteriors/dl']), axis=1)))
    sigma_z = 1.0/sqrt(sum(1/var(log(f['posteriors/m1det']), axis=1)))
sigma_lnH0 = 0.01
sigma_Om = 0.01

Here we return the covariance matrix:

In [33]:
def make_cov(sigma_lndL, sigma_z, sigma_lnH0, sigma_Om):
    grad_d = [-1.0/Planck15.H0.to(u.km/u.s/u.Mpc).value, dlndLdOm, dlndLdw]
    F_d = 1/sigma_lndL**2*np.outer(grad_d, grad_d)
    F_z = 1/sigma_z**2
    
    F = zeros((4,4))
    F[:3,:3] = F_d
    F[3,3] = F_z
    
    F[0,0] += 1.0/(Planck15.H0.to(u.km/u.s/u.Mpc).value*sigma_lnH0)**2
    F[1,1] += 1.0/sigma_Om**2
    
    return np.linalg.inv(F)

For the canonical 1% $H_0$, 1% $\Omega_M$ measurement:

In [49]:
C = make_cov(sigma_lndl, sigma_z, 0.01, 0.003)
print(sqrt(diag(C)))
print('sigma_w = {:.3f}'.format(sqrt(diag(C))[2]))

[0.6774     0.003      0.06623204 0.00136655]
sigma_w = 0.066


For a more precise $H_0$, at the 0.5% level:

In [50]:
C = make_cov(sigma_lndl, sigma_z, 0.005, 0.003)
print(sqrt(diag(C)))
print('sigma_w = {:.3f}'.format(sqrt(diag(C))[2]))

[0.3387     0.003      0.0362396  0.00136655]
sigma_w = 0.036


And with a perfect $H_0$ measurement:

In [51]:
C = make_cov(sigma_lndl, sigma_z, 0.00001, 0.003)
print(sqrt(diag(C)))
print('sigma_w = {:.3f}'.format(sqrt(diag(C))[2]))

[0.0006774  0.003      0.01699572 0.00136655]
sigma_w = 0.017


With perfect $H_0$ and $\Omega_M$:

In [52]:
C = make_cov(sigma_lndl, sigma_z, 0.00001, 0.00003)
print(sqrt(diag(C)))
print('sigma_w = {:.3f}'.format(sqrt(diag(C))[2]))

[6.77400000e-04 3.00000000e-05 1.56643999e-02 1.36655498e-03]
sigma_w = 0.016
